**Содержание**<a id='toc0_'></a>    
- [Свёртка, каскад свёрток](#toc1_)    
    - [Свертка](#toc1_1_1_)    
    - [Пуллинг](#toc1_1_2_)    
- [Собери их всех: архитектура LeNet (1998)](#toc2_)    
- [Собери их все: AlexNet (2012) и VGG (2014)](#toc3_)    
  - [AlexNet](#toc3_1_)    
    - [Почему ReLU. Потому что затухание градиента](#toc3_1_1_)    
  - [VGG](#toc3_2_)    
- [Собери их все: GoogLeNet и ResNet (2015)](#toc4_)    
  - [Inception block](#toc4_1_)    
    - [Bottle neck](#toc4_1_1_)    
  - [GoogLeNet](#toc4_2_)    
  - [Residual Block](#toc4_3_)    
  - [ResNet](#toc4_4_)    
- [Итого](#toc5_)    
- [Теоретические задачи: архитектуры сверточных нейронных сетей](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Свёртка, каскад свёрток](#toc0_)

Почему нельзя обойтись полносвязной сетью, ведь

- сигмоидные полносвязанные нейронные сети полны. При помощи них можно восстановить любую зависимость с любой точностью

Потому что 
- у нас ограниченные вычислительные возможности
  - если полносвязная сеть научилась аппроксимировать (сигмоидами) векторное представление кота по центру кадра, то кота в углу кадра она все равно не увидит
  - поэтому нужны обучающие образцы для всех возможных размещений объекта в кадре
  - или нужны дополнительные маски для кодирования размещения объектов в кадре
- нужно использовать дополнительную информацию о структуре данных
  - часто требуется использовать информацию о структуре данных, то есть - "какой пиксель находится рядом с каким"

**Операция свертки инварианта для расположения объекта в кадре**

### <a id='toc1_1_1_'></a>[Свертка](#toc0_)

Ядром свертки проходим по данным. Ядро множится по-элементно на фрагмент, результат суммируется, если это нейросеть - еще добавляется смещение, записывается в сверточную матрицу такой же или меньшей (или большей, если паддинг больше половины ядра) размерности.
- есть несколько основных параметров (**padding, strides, dilation**)
- [идеальная иллюстрация](https://github.com/vdumoulin/conv_arithmetic)

Пусть матрица признаков равна [[4, 2, -1],[-6, 0, 5],[3, 2, 2]], а ядро свертки -- [[0, 1, 2],[1, -1, 0],[1, 0, -2]]

Каков будет результат применения свертки со `stride=2, padding=1`

\*) 2д свертка значит матрица ядра двухмерная

**Примерное АПИ сверток торча:**

In [50]:
import torch

# добавляется 1 измерение для каналов, 1 измерение для батчей
X = torch.tensor([[4, 2, -1],[-6, 0, 5],[3, 2, 2]]).float().unsqueeze(0).unsqueeze(0)

ker = torch.tensor([[0, 1, 2],[1, -1, 0],[1, 0, -2]]).float()

# добавляется 1 измерение для ВХОДНЫХ каналов, 1 измерение для ВЫХОДНЫХ каналов
fltr = ker.unsqueeze(0).unsqueeze(0)

conv = torch.nn.Conv2d(in_channels=1, 
                       out_channels=1, 
                       kernel_size=ker.shape, 
                       padding=1, 
                       stride=2, 
                       bias=False)              # для порядка аргументы по именам

conv.weight.data = torch.nn.Parameter(fltr)     # для порядка, обычно это настраиваевоеваемый параметр

conv(X)

tensor([[[[-4.,  3.],
          [-9.,  5.]]]], grad_fn=<ConvolutionBackward0>)

Когда мы работаем с изображениями -- изображения, как правило, трёхканальные. Для каждого из этих цветов есть некоторая матрица, которая описывает, сколько зелёного, красного и синего цвета в каждом пикселе. Как свёртка работает в этом случае? В случае с трёхканальными изображениями ядро свёртки -- это трёхмерная табличка, а не двумерная, как в предыдущем случае.

**3 входных канал - 1 фильтр (3 канальный, т.е. 3 ядра) - 1 выходной канал**

<img src="./img/conv3d.png" width="500">

Например: в свертке размером 3х3, которая применяется к трехканальному изображению (не считая слой активации, не учитывая bias) будет 27 параметров: 3х3 для каждого из трех каналов

Следующие сверточные слои могут быть более витееватыми

**N входных каналов - M фильтров (N канальных) - M выходных каналов**

<img src="./img/NMconv3d.png" width="500">

N входных каналов - это уже не обязательно цветовые каналы, а может быть результаты предыдущего слоя сверток.

Например: пусть размер матрицы признаков = 4(высота)х5(ширина)x2(кол-во каналов), размеры ядра свертки = 3x3, stride=2, padding=1, количество выходных фильтров = 8. Один 2-канальный фильтр даст матрицу 2х3, таких фильтров 8 -> 6*8=48

    "Прежде чем мы углубимся в это, я просто хочу провести четкое различие между терминами «ядро» и «фильтр», потому что я видел, что многие люди используют их взаимозаменяемо. Ядро, как описано ранее, представляет собой матрицу весов, которые умножаются на входные данные для извлечения соответствующих признаков. Размеры матрицы ядра как свертка получает свое имя, Например, в двумерных свертках матрица ядра является двумерной матрицей.

    Однако фильтр представляет собой объединение нескольких ядер, каждое из которых назначено определенному каналу ввода. Фильтры всегда на одно измерение больше, чем ядра. Например, в двумерных свертках фильтры являются трехмерными матрицами (что по существу является объединением двумерных матриц, то есть ядер). Таким образом, для слоя CNN с размерами ядра h * w и входными каналами k размеры фильтра k * h * w".

### <a id='toc1_1_2_'></a>[Пуллинг](#toc0_)

Проследив принцип работы биологического мозга, учёные смогли разработать математический аппарат для извлечения свойств.
Оценив общее количество слоёв и свойств, которые нужно проанализировать для отслеживания сложных геометрических форм, можно показать, что количество необходимых вычислительных ресурсов растёт экспоненциально вместе с ростом количества свойств. Для решения этой проблемы была разработана методика пулинга (pooling). Её идея очень проста: если некая область содержит ярко выраженные свойства, то мы можем отказаться от поиска других свойств в этой области (с)

Часто используется **max-pooling** - данные делятся на подобласти и выбираются максимумы. 

Часто для этого выбирается фильтр 2х2:

$${\displaystyle f_{X,Y}(S)=\max _{a,b=0}^{1}S_{2X+a,2Y+b}.}$$

**Для обратного прохода**: градиент равен 1, обратный проход только через ячейку с максимумом

Часто испольщуемым вариантом макспуллинга является `RoI-pooling` (Region of Interest), где пуллинг выполняется только в заданном участке данных, поделенном на заданные подучастки (такое деление - это гиперпараметр метода), например это может быть некоторая область кадра и некоторые ее подобласти.

\*) Кроме того, может использоваться и другая метрика для пуллинга (минимум, среднее, $L_2$-норма), но это редко и часто хуже работает.

**Пуллинг выполняется для каждого канала независимо, количество каналов не меняется!**

Мы исходим из того, что большие значения после свертки несут большую часть информации, поэтому выкидываем 3/4 информации как шум (для пулинга 2х2)

# <a id='toc2_'></a>[Собери их всех: архитектура LeNet (1998)](#toc0_)

Предложена для распознавания рукописных цифр. LeNet решает задачу, которая называется MNIST. Эта задача заключается в том, что нам нужно верно классифицировать на десять классов рукописные цифры (от 0 до 9).

<img src="./img/lenet.png" width="700">

- Вход: 32х32, 1 канал

1 слой:
- Свертка: 5х5, 6 выходных каналов, без паддинга, шаг 1 (stride)
  - 6х28х28
- Макс-пуллинг: 2х2
  - 6х14х14
- Активация тангенсом (-1, 1)
  - 6х14х14

2 слой:
- Свертка: 5х5, 16 выходных каналов, без паддинга, шаг 1 (stride)
  - 16х10х10
- Макс-пуллинг: 2х2
  - 16х5х5
- Активация тангенсом (-1, 1)
  - 16х5х5

3 вытягивание в строку (flatten - одномеризация): 16х5х5 -> 1x1x400

Пачка полносвязных слоев:
- 400х120 + тангенс
- 120х84 + тангенс
- 84х10 + софтмакс

Получили 10 вероятностей каждого класса (цифр)

**Обучение:**

Что обучается в сверточных сетях?
- Обучаются фильтры сверток и их смещения
- Обучаются полносязные слои (матрицы линейного преобразования: веса/смещения)

Функция потерь:
- кросс-энтропия, т.к. у нас вероятности классов

Оптимизатор:
- SDGM или Adam
  - lr=3e-4, $\beta_1 =0.9$, $\beta_1 =0.999$

<img src="./img/lenet2.png" width="700">

# <a id='toc3_'></a>[Собери их все: AlexNet (2012) и VGG (2014)](#toc0_)

MNIST это скорее чисто учебная задача.

Более сложная задача **ImageNet 1000**. Это довольно большая база данных с изображениями (15 млн.), которые нужно классифицировать на 1000 классов. База шумная, фактически только 1.5 млн. действительно относятся к этим 1000 классов.

Эта задача часто рассматривается как референсная для оценки качества нейросети.

## <a id='toc3_1_'></a>[AlexNet](#toc0_)

<img src="./img/alexnet.png" width="700">

- Вход: 224х224, 3 канала
  - если ч/б, то один канал просто дублируется 3 раза
  - если разрешение !=224х224, то кадр масштабируется билинейной интерполяцией до 256х256 и берется случайный фрагмент 224х224
  - в некоторых источниках пишут 227х227

Сверточные слои:
- Свертка: 11х11, 96 выходных каналов, шаг 4 (stride)
  - паддинг (подбирается по принятому размеру входа, чтоб получилось 55: 5 если 227, 2 если 224)
  - 96х55х55
- Свертка: 5х5, 256 выходных каналов, шаг 1, паддинг 2 + макс-пуллинг 2х2
  - паддинг - аналогично, чтоб на выходе было 27х27
  - 256х27х27
- Свертка: 3х3, 384 выходных каналов, шаг 1, паддинг 1 + макс-пуллинг 2х2
  - паддинг - аналогично, чтоб на выходе было 13х13
  - 384х13х13

- **Свертка: 3х3**, 384 выходных каналов, шаг 1, паддинг 1
  - паддинг - аналогично, чтоб на выходе было 13х13
  - 384х13х13
- **Свертка: 3х3**, 256 выходных каналов, шаг 1, паддинг 1
  - паддинг - аналогично, чтоб на выходе было 13х13
  - 256х13х13

\*) после каждого сверточного слоя м.б. еще дополнительно функция активации **ReLU**, а может и не быть...

- Макс-пуллинг с шагом 2 (страйд)
  - 256х6х6

- flatten - одномеризация: 256х6х6 -> 1x1x9216

Пачка полносвязных слоев:
- 9216х4096 + Relu
- 4096х4096 + Relu
- 4096х1000 + Softmax

Получили 1000 вероятностей каждого класса

**Обучение:**

Функция потерь:
- кросс-энтропия, т.к. у нас вероятности классов

Оптимизатор:
- SDGM или Adam

**Фишка:**
- свертки 3х3 без макс-пуллинга были предложены в оригинальной статье и обусловлены тем, что у авторов комп не потянул в этом месте еще одну свертку 5х5. Вместо этого сделали каскад сверток 3х3. Так они решили расширить рецептивное поле сверток и так и осталось.
  - Свертка 5х5 это 5x5+bias=26 обучаемых параметров, две свертки 3х3 это 3х3+1+3х3+1=20 обучаемых параметров

### <a id='toc3_1_1_'></a>[Почему ReLU. Потому что затухание градиента](#toc0_)

$$\sigma(x) = \frac{1}{1+e^{-x}}\\
\sigma' = \sigma(1-\sigma) \leq \frac{1}{4} \to 0$$

Если у нас сеть из $n$ сигмоид, то производная фукнции потерь ограничена $\frac{1}{4}^n$ и с учетом ограниченной вычислительной точности все быстро сравнивается с 0. Это называется **затухание градиента**. Глубокие слои перестают обучаться, т.к. производная до них просто не доходит при обратном распространении.

Частично это решается гипертангенсом (иногда его тоже зовут сигмоидой):

$$\sigma^*(x) = \frac{e^{x}-e^{-x}}{e^{x}-e^{-x}}\\
\sigma' = (1+\sigma)(1-\sigma) \leq 1$$

- частично, потому что областях плато (-1, 1) производная $\to 0$
- при обратном распространении ошибки такие производные все равно зануляют градиент

*Почему бы тогда не брать просто линейные функции активации?* А потому, что многослойная сеть тогда будет эквивалентна однослойной полносвязной сети, т.е. просто одному линейному преобразованию.

*Почему бы тогда просто не домножать градиент просто на достаточно большое число, если он уходит в 0?* Тогда это не будет градиент нужной нам функции потерь, а просто некоторое направление, которое возможно и будет коррелировать с истинным градиентом, но не факт.

**Что дает ReLU (rectified linear unit)**

$$ReLU(x) = \begin{cases} 
0, & x < 0 \\
x, & x \geq 0
\end{cases}$$

Как вариант ELU (exponential linear unit)
$$ELU(x) = \begin{cases} 
e^x - 1, & x < 0 \\
x, & x \geq 0
\end{cases}$$

Как вариант Leaky ReLU
$$Leaky-ReLU(x) = \begin{cases} 
\alpha x, & x < 0 \\
x, & x \geq 0
\end{cases}$$

Как вариант SeLU
$$SeLU(x) = \begin{cases} 
\alpha e^x - \alpha, & x < 0 \\
x, & x \geq 0
\end{cases}$$

- обучаемый параметр $\alpha$

Каскад таких функций будет обращаться в 0 только если производная =0. Но если сеть достаточно глубокая, то вероятность, что на каком-то из слоев градиент затухнет и перестанет распространяться дальше, все равно довольно высокая, хотя такой прямой зависимости от количества слоев, как у сигмоиды, тут уже нет.

## <a id='toc3_2_'></a>[VGG](#toc0_)

По названию компании Visual Geometry Group

<img src="./img/vgg.png" width="500">

Суть в том, что чтобы избежать затухания градиента и долгого обучения сначала обучается простая модель A, потом к ней добавляются слой B. 

Если градиет затухнет, то глубокие слои уже обучены. 

**Фишки:**

- В слое С добавляется фиктивная свертка 1х1, чтобы остальные уже обученные параметры сети адаптировались, что есть еще слои. Потом он заменяется на свертку 3х3, которая дообучается уже сама по себе

- Для увеличения рецептивного поля используются каскады сверток между макс-пуллингами

Часто такая архитектура используется в качестве базовой (backbone) для решения других задач, не связанных с ImageNet, путем добавления и замены выходных слоев

# <a id='toc4_'></a>[Собери их все: GoogLeNet и ResNet (2015)](#toc0_)

## <a id='toc4_1_'></a>[Inception block](#toc0_)

<img src="./img/inception.png">

Идея inception блока состоит в том, что возможно, что мы хотим сделать не одну операцию к входу, например, свёртку, а несколько разных и посмотреть, каков будет результат их применения.

Inception блок делает параллельно несколько операций. Главное, чтобы результат всех параллельных блоков операций имел одинаковый выходной размер (например, кадр размером 128х128). Количество выходных каналов может быть разным. 

После этого все результаты всех параллельных блоков конкатенируются, получается тензор размером (сумма выходных каналов разных преобразований x размер кадра)

### <a id='toc4_1_1_'></a>[Bottle neck](#toc0_)

В примере мелькают свертки 1х1. Это прием для уменьшения количества каналов, если нас устраивает примерный результат. Например вместо свертки 5х5 с 2048 вх/вых.каналами можно пройтись сверткой 1х1 с 512 вых.каналами (уменьшить число каналов в 4 раза), а потом пройтись сверткой 5х5 с 512 вх. и 2048 вых. каналами. Число операций уменьшается в 4 раза. Результат обычно почти не отличается на больших размерностях.

Например, если у нас имеется кадр RGB с тремя каналами, и мы к нему применим свёртку 1х1 с 1 выходным каналом,  то в результате получим кадр того же размера, но с одним каналом. В нём показатели RGB некоторой лин.комбинацией входных каналов (зависит от ядра свертки). Для таких малых размерностей смысла нет делать bottle neck.

К свертке 1х1 может добавляться также функция активации, что эквивалентро применению этой функции ко всем пикселям кадра.

Т.о. свёртка 1х1 - это просто умножение всех пикселей на одинаковый вес и добавление смещения (сеть может выучить нормализацию) и пропускание выхода через нелинейность. Это также помогает получить меньше кода, чем если писать все эти операции по-отдельности - просто сделать свёртку 1х1.

## <a id='toc4_2_'></a>[GoogLeNet](#toc0_)

Сеть, состоящая из Inception blocks

<img src="./img/googlenet.png">

Сеть достаточно глубокая. И чтобы градиент не затухал к слоям подключены дополнительные вспомогательные выходные слои, от которых распространяется дополнительный градиент функции ошибок.
- для значений истинных классов в дополнительных выходах используются те же метки, что и у основного.

В оригинальной архитектуре таких вспомогательных выходных слоев два (кроме основного).

## <a id='toc4_3_'></a>[Residual Block](#toc0_)

Придумали в миклосохте

<img src="./img/resblock.png">

Вход в некоторую свертку $x$ обрабатывается функцией $F(x)$ и на выход из блока выдается их сумма $F(x)+x$. То есть добавляется связь в обход свертки (residual connection).

$$ y = F(x) + x \\
y' = F'(x) + 1 \\

\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial x} = \frac{\partial L}{\partial y}[F'(x) + 1 ]$$

Т.е. градиент не затухает, информация проходит частично в обход.

Тут также часто применяют свертки 1х1 для экономии вычислений

## <a id='toc4_4_'></a>[ResNet](#toc0_)

<img src="./img/resnet.png">

Схема оригинальной архитектуры для ImageNet

На схеме ResNet блоки, в которых происходит уменьшение размера картинки в два раза, нарисованы с пунктирной дугой. Как там происходит прибавление входного х к выходному f(x)? Эти тензоры имеют разный shape, поэтому
- есть несколько способов уменьшить на этих residual connections размер в два раза. Один из вариантов - сделать свертку Conv2d(C, 2C, kernel_size=(1, 1), stride=(2, 2)). Есть и много других. Ну, например, можно сделать average pooling 2x2 и дважды продублировать каналы.

**Фишка:**
- на выходе нет полносвязных слоев, т.к. сеть достаточно глубокая и предполагается, что оптимальная линейная комбинация активационных функций уже найдена в виде сверточных слоев и обходных связей (очень глубокие сети имеют тенденцию отлично дискриминировать объекты и без FC-слоёв).
- для решения конкретных задач могут добавляться выходные полносвязные слои, например для ImageNet1000 это будет AvgPooling + FC1000

Модификации сети отличаются количеством слоев, распространенные конфигурации 
- Resnet-18, 
- ResNet-32, 
- ... 
- ResNet-1024

Обычно вход такой сети - кадры 224х224. Это исторически сложившийся размер, к которому часто приводят другие размеры кадра.
- но можно подавать любой размер, желательно кратный 32, т.к. 5 раз происходит сжатие кадра в 2 раза (слои /2 на схеме), либо отбрасывать лишнее или дополнять 0 до кратного 32

ResNet уже неважно, сколько пикселей во входном изображении, важен масштаб деталей.
- в конце сети тензор усредняется и таким образом масштабируется до размера полносвязанного слоя
- мы усредняемся и не важно по какой базе (7х7, 32х32 да хоть 1024х1024, сколько дошло до конца от размера исходного кадра) важно то, что мы всё свели постоянному количеству 512 выходных каналов.

# <a id='toc5_'></a>[Итого](#toc0_)

LeNet (1998)
- функция активации tanh, свёрточные слои и пулинг

AlexNet (2012)
- каскад свёрток (увеличение receptive field) и функция активации ReLU

VGG (2014)
- обучать урезанную версию сети, постепенно добавляя слои, а также свёртки 1х1

GoogLeNet (2015)
- слои, где собирается тензор из нескольких свёрток разного размера

ResNet (2015)
- обходные соединения, пробрасывающие градиент ошибки в обход свёртки

# <a id='toc6_'></a>[Теоретические задачи: архитектуры сверточных нейронных сетей](#toc0_)

Рассмотрим некоторую нейронную сеть $y = f(x)$, где $y$ и $x$ - тензоры.

Пусть тензор $x$ имеет размер $N \times C \times H \times W$, а тензор $y$ имеет размер $N \times C' \times H' \times W'$.

Рассмотрим некоторый подтензор : $\texttt{y[n, :, a, b]}$, где $\texttt{n, a, b}$ - некоторые координаты в тензоре 

$$y: 0 \leq \texttt{n} \leq N - 1,\\ 0 \leq \texttt{a} \leq H' - 1, \\ 0 \leq \texttt{b} \leq W'-1$$

Этот вектор зависит от некоторого подтензора входного тензора $x: \texttt{x[n, :, a':a'+h, b':b'+w]}$. То есть, если поменять хотя бы одно значение в этом подтензоре, то результат может измениться, вне этого подтензора любые изменения не изменят вектор $\texttt{y[n, :, a, b]}$.

Размеры этого тензора h, w называются receptive field или **пятном восприятия**.

Найдите receptive field для следующей подсети:

    torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(3, 1)),
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(1, 3))
    )

**RF = (3, 3)**

    torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(3, 3)),
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(3, 3))
    )

**RF = (5, 5)**

    torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(5, 5), stride=(2, 2)),
        torch.nn.Conv2d(in_channels=C, out_channels=C, kernel_size=(5, 5))
    )

**RF = (13, 13)**

Рассмотрим две нейронные сети:

1:

    torch.nn.Sequential(
        torch.nn.Conv2d(2 * С, 2 * С, kernel_size=(3, 3), padding=1, bias=False)
    )

2:

    torch.nn.Sequential(
        torch.nn.Conv2d(2 * С, С, kernel_size=(1, 1), bias=False),
        torch.nn.Conv2d(С, С, kernel_size=(3, 3), padding=1, bias=False),
        torch.nn.Conv2d(С, 2 * С, kernel_size=(1, 1), bias=False)
    )
    
Найдите отношение количества параметров первой сети к количеству параметров во второй сети.

Убедитесь в том, что количество операций для некоторого входного тензора размером $N \times 2C \times H \times W$ соотносится так же.

In [19]:
import torch

C = 1
c1 = torch.nn.Sequential(
        torch.nn.Conv2d(2 * C, 2 * C, kernel_size=(3, 3), padding=1, bias=False)
    )
    
list(c1.parameters())   # 3*3*4

c2 = torch.nn.Sequential(
        torch.nn.Conv2d(2 * C, C, kernel_size=(1, 1), bias=False),
        torch.nn.Conv2d(C, C, kernel_size=(3, 3), padding=1, bias=False),
        torch.nn.Conv2d(C, 2 * C, kernel_size=(1, 1), bias=False)
    )


list(c2.parameters())   # 2 + 3*3 + 2

3*3*4 / (2 + 3*3 + 2)

2.769230769230769

Рассмотрим граф вычисления, который состоит из $N$ блоков, следующих друг за другом:

![alt text](./img/task.svg)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Найдите производную $\frac {\partial L}{\partial x_0}$, если известны все производные $\frac{\partial f_{i}}{\partial x_i}$.

$\frac{\partial L}{\partial x_0} = \frac{\partial L}{\partial x_1} \frac{\partial x_1}{\partial x_0}  = \frac{\partial L}{\partial x_1} \frac{\partial f_0}{\partial x_0} \\
...\\
\frac{\partial L}{\partial x_0} = \frac{\partial L}{\partial x_N} \prod_{i=0}^N \frac{\partial f_i}{\partial x_i} $

Рассмотрим граф вычисления, который состоит из $N$ блоков, следующих друг за другом следующего вида:

![alt text](./img/task_res.png)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Найдите производную $\frac {\partial L}{\partial x_0}$, если известны все производные $\frac{\partial f_{i}}{\partial x_i}$.


$\frac{\partial L}{\partial x_0} = \frac{\partial L}{\partial x_1} \frac{\partial x_1}{\partial x_0}  = \frac{\partial L}{\partial x_1} \frac{\partial (f_0 + x_0)}{\partial x_0} =  \frac{\partial L}{\partial x_1} (\frac{\partial f_0}{\partial x_0} + 1)\\
...\\
\frac{\partial L}{\partial x_0} = \frac{\partial L}{\partial x_N} \prod_{i=0}^N (\frac{\partial f_i}{\partial x_i} + 1) $

Рассмотрим граф вычисления, который состоит из $N$ блоков, следующих друг за другом:

![alt text](./img/task.svg)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Допустим, что все производные $\left| \frac{\partial f_{i}}{\partial x_i}\right| \leq 0.5$. Найдите, каким значением будет ограничено сверху значение $\left|\frac{\partial L}{\partial x_{0}}\right|$.

$$|\frac{\partial L}{\partial x_0}| \leq |\frac{\partial L}{\partial x_N}| 0.5^N$$

Рассмотрим граф вычисления, который состоит из $N$ блоков, следующих друг за другом:

![alt text](./img/task_res.png)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Допустим, что все производные $\left| \frac{\partial f_{i}}{\partial x_i}\right| \leq 0.5$. Найдите, каким значением будет ограничено сверху значение $\left|\frac{\partial L}{\partial x_{0}}\right|$.

$$|\frac{\partial L}{\partial x_0}| \leq |\frac{\partial L}{\partial x_N}| 1.5^N$$

Рассмотрим граф вычисления, который состоит из NN блоков, следующих друг за другом:

![alt text](./img/task.svg)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Пусть существует ненулевая вероятность того, что $\frac{\partial f_{i}}{\partial x_i} = 0$ и она равна $p$. При этом, во всех остальных случаях $\frac{\partial f_{i}}{\partial x_i} = 1$ (ситуация очень похожа на применение ReLU).

Найдите вероятность того, что $\frac{\partial L}{\partial x_0} = 0 $.

Вероятность 1 и более "успехов" в схеме Бернулли с N испытаниями:

- равна 1 - вероятность 0 "успехов" (ровно 0 "неудач") равна $1 - (1-p)^N$

Рассмотрим граф вычисления, который состоит из NN блоков, следующих друг за другом:


![alt text](./img/task_res.png)

Допустим, что мы знаем производную лосс-функции $L$ по выходу из последнего блока $x_N$ ($\frac{\partial L}{\partial x_N}$). Пусть существует ненулевая вероятность того, что $\frac{\partial f_{i}}{\partial x_i} = 0$ и она равна $p$. При этом, во всех остальных случаях $\frac{\partial f_{i}}{\partial x_i} = 1$ (ситуация очень похожа на применение ReLU).

Найдите вероятность того, что $\frac{\partial L}{\partial x_0} = 0 $.

- это невозможное событие, поэтому вероятность = 0.

**Семинар** про классификацию рукописных цифр сверточной сетью в `./Neural_Networks_and_CV`